In [ ]:
import menpo3d.io as m3io
import menpo.io as mio
from menpo.shape import PointCloud

<h3> Load the data (Mesh and Landmarks) </h3>

In [ ]:
mesh = m3io.import_mesh('../menpo3d/data/james.obj')
lms = m3io.import_landmark_file('../menpo3d/data/james.ljson')['LJSON']

<h3> Show the mesh </h3>

In [ ]:
# Default values for TriMesh, TextureMesh viewer are
# figure_id None
# new_figure True
# in that case an automatic figure_id will be given
# with 'Figure_{n}' format
# n will be an increased integer starting from zero

mesh.view(inline=True,) #wait a bit before magic happens

<h3> Show the mesh and landmarks </h3>

In [ ]:
mesh.view(inline=True, figure_id='James')

In [ ]:
# Add landmarks to figure with id James
lms_poincloud = PointCloud(lms.points)
lms_poincloud.view(inline=True, figure_id='James',new_figure=False)

In [ ]:
# Add landmarks to figure with id Figure_0
lms_poincloud.view(inline=True, figure_id='Figure_0', new_figure=False)

<h3> Show a mesh that has landmarks </h3>

In [ ]:
mesh.landmarks = lms

In [ ]:
# the new figure will have figure_id Figure_1
# Note the difference between plotting a pointcloud
# and landmarks 
mesh.landmarks.view(inline=True, new_figure=True)

In [ ]:
# The mesh has now landmarks, so they would be plotted as well
# the figure id is now Figure_2
mesh.view(inline=True)

<h3> Load a Morphable Model </h3>

In [ ]:
# Load model and its landmarks indices 
model =  mio.import_pickle('../menpo3d/data/3DMD_all_all_all_10.pkl')['model']
lms_indices = [21868, 22404, 22298, 22327, 43430, 45175, 46312, 47132, 47911, 48692,
               49737, 51376, 53136, 32516, 32616, 32205, 32701, 38910, 39396, 39693,
               39934, 40131, 40843, 41006, 41179, 41430, 13399, 8161, 8172, 8179, 8185,
               5622, 6881, 8202, 9403, 10764, 1831, 3887, 5049, 6214, 4805, 3643, 9955,
               11095, 12255, 14197, 12397, 11366, 5779, 6024, 7014, 8215, 9294, 10267,
               10922, 9556, 8836, 8236, 7636, 6794, 5905, 7264, 8223, 9063, 10404, 8828,
               8228, 7509]

In [ ]:
model.view(inline=True, figure_id='Model', n_parameters=10, landmarks_indices=lms_indices)

In [ ]:
model.view(widget_style='')

<h3> Fail cases </h3> 

In [ ]:
# It should fail if the previous cells have been executed, as default values for landmarker viewer are
# figure_id = None and new_figure=False, so it could not
# find a figure with id None
lms.view(inline=True,)

In [ ]:
# It should fail if the previous cells have been executed, as we have already had a figure with id 
# James and we cannot create a new one with the same figure_id
mesh.view(inline=True, figure_id='James', new_figure=True)

In [ ]:
# It should fail if the previous cells have been executed, as we have already had a figure with id 
# Model and we cannot create a new one with the same figure_id
model.view(inline=True, figure_id='Model')

<h3> Testing </h3>

In [ ]:
from ipywidgets import Widget
from menpo3d.visualize.viewk3dwidgets import K3dwidgetsRenderer

In [ ]:
for x in Widget.widgets.values():
   # print(type(x),x.model_id)
    #if isinstance(x, K3dwidgetsRenderer):
    if hasattr(x,'figure_id'):
        print(type(x),x.model_id, x.figure_id)